In [4]:
!pip install pandas

  Using cached https://files.pythonhosted.org/packages/c3/e2/00cacecafbab071c787019f00ad84ca3185952f6bb9bca9550ed83870d4d/pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/60/2e/dec1cc18c51b8df33c7c4d0a321b084cf38e1733b98f9d15018880fb4970/pytz-2022.1-py2.py3-none-any.whl


In [6]:
import requests
import json
from datetime import datetime 
import os
import pandas as pd
#import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from joblib import dump

In [35]:
def get_weather():

    ville=['paris','london','washington']
    for city in ville:
# My API key is defined in my config.py file.
        paramaters = {'q':city , 'appid':'62ee8788e06f7ba0e7bd21472c4b16df'}

        result     = requests.get("http://api.openweathermap.org/data/2.5/weather?", paramaters)
    #25237a5210330ad588a2a33c388da468
    # If the API call was sucessful, get the json and dump it to a file with 
    # today's date as the title.
        if result.status_code == 200 :
    # Get the json data 
            json_data = result.json()
    # print(json_data)

    # create directory
            createDirectory()

    # make file 
            filename = "raw_files/" + str(datetime.now()) + '.json'    

            with open(filename, 'w') as file:
                print(filename)
                json.dump(json_data, file)
                print(json_data)
                print("closing file . . . ")            

        else :
            print ("Error In API call.")

        
def createDirectory():
    # creating directory 
    dirName = "raw_files"
    
    # create target directory if don't exists    
    if not os.path.exists(dirName):
    # exception handeling for file not found
        try:
            os.makedirs(dirName)
            print("Directory ", dirName, "Created")

        except FileExistsError as fe:
            print("Error : ", fe)
            print("Directory ", dirName, "already exists")


In [36]:
get_weather()

raw_files/2022-06-02 17:45:21.749105.json
{'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 289.07, 'feels_like': 288.23, 'temp_min': 286.48, 'temp_max': 291.03, 'pressure': 1016, 'humidity': 58}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 50}, 'clouds': {'all': 0}, 'dt': 1654216955, 'sys': {'type': 1, 'id': 6550, 'country': 'FR', 'sunrise': 1654228240, 'sunset': 1654285628}, 'timezone': 7200, 'id': 2988507, 'name': 'Paris', 'cod': 200}
closing file . . . 
raw_files/2022-06-02 17:45:21.817470.json
{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 284.98, 'feels_like': 284.44, 'temp_min': 282.64, 'temp_max': 286.47, 'pressure': 1019, 'humidity': 85}, 'visibility': 10000, 'wind': {'speed': 2.57, 'deg': 50}, 'clouds': {'all': 6}, 'dt': 1654216456, 'sys

In [49]:
def transform_data_into_csv(n_files=None, filename='data.csv'):
    parent_folder = 'raw_files'
    files = sorted(os.listdir(parent_folder), reverse=True)
    if n_files:
        files = files[:n_files]

    dfs = []

    for f in files:
        with open(os.path.join(parent_folder, f), 'r') as file:
            data_temp = json.load(file)
            dfs.append(
                {
                    'temperature': data_temp['main']['temp'],
                    'city': data_temp['name'],
                    'pression': data_temp['main']['pressure'],
                    'date': f.split('.')[0]
                }
            )

    df = pd.DataFrame(dfs)

    print('\n', df.head(10))

    df.to_csv(os.path.join('clean_data', filename), index=False)


In [50]:
transform_data_into_csv(n_files=20, filename='data.csv')


    temperature        city  pression                 date
0       291.34       Paris      1016           2022-06-02
1       294.53  Washington      1008  2022-06-02 17:45:21
2       284.98      London      1019  2022-06-02 17:45:21
3       289.07       Paris      1016  2022-06-02 17:45:21
4       294.53  Washington      1008  2022-06-02 17:45:19
5       284.98      London      1019  2022-06-02 17:45:19
6       289.29       Paris      1016  2022-06-02 17:45:19
7       294.53  Washington      1008  2022-06-02 17:45:17
8       284.98      London      1019  2022-06-02 17:45:17
9       289.29       Paris      1016  2022-06-02 17:45:17


In [51]:
transform_data_into_csv(n_files=None, filename='fulldata.csv')


    temperature        city  pression                 date
0       291.34       Paris      1016           2022-06-02
1       294.53  Washington      1008  2022-06-02 17:45:21
2       284.98      London      1019  2022-06-02 17:45:21
3       289.07       Paris      1016  2022-06-02 17:45:21
4       294.53  Washington      1008  2022-06-02 17:45:19
5       284.98      London      1019  2022-06-02 17:45:19
6       289.29       Paris      1016  2022-06-02 17:45:19
7       294.53  Washington      1008  2022-06-02 17:45:17
8       284.98      London      1019  2022-06-02 17:45:17
9       289.29       Paris      1016  2022-06-02 17:45:17


In [52]:

def compute_model_score(model, X, y):
    # computing cross val
    cross_validation = cross_val_score(
        model,
        X,
        y,
        cv=3,
        scoring='neg_mean_squared_error')

    model_score = cross_validation.mean()

    return model_score


def train_and_save_model(model, X, y, path_to_model='./app/model.pckl'):
    # training the model
    model.fit(X, y)
    # saving model
    print(str(model), 'saved at ', path_to_model)
    dump(model, path_to_model)


def prepare_data(path_to_data='./app/clean_data/fulldata.csv'):
    # reading data
    df = pd.read_csv(path_to_data)
    # ordering data according to city and date
    df = df.sort_values(['city', 'date'], ascending=True)

    dfs = []

    for c in df['city'].unique():
        df_temp = df[df['city'] == c]

        # creating target
        df_temp.loc[:, 'target'] = df_temp['temperature'].shift(1)

        # creating features
        for i in range(1, 10):
            df_temp.loc[:, 'temp_m-{}'.format(i)
                        ] = df_temp['temperature'].shift(-i)

        # deleting null values
        df_temp = df_temp.dropna()

        dfs.append(df_temp)

    # concatenating datasets
    df_final = pd.concat(
        dfs,
        axis=0,
        ignore_index=False
    )

    # deleting date variable
    df_final = df_final.drop(['date'], axis=1)

    # creating dummies for city variable
    df_final = pd.get_dummies(df_final)

    features = df_final.drop(['target'], axis=1)
    target = df_final['target']

    return features, target


In [53]:

#if __name__ == '__main__':
def find_score():

    X, y = prepare_data('clean_data/fulldata.csv')

    score_lr = compute_model_score(LinearRegression(), X, y)#a
    score_dt = compute_model_score(DecisionTreeRegressor(), X, y)#b
    score_rf = compute_model_score(RandomForestRegressor(), X, y)#c


    if score_lr>=score_dt:
        if score_lr >= score_rf:
            train_and_save_model(
            LinearRegression(),
            X,
            y,
            'clean_data/best_model.pickle'
        )
        else:
            train_and_save_model(
            RandomForestRegressor(),
            X,
            y,
            'clean_data/best_model.pickle'
        )
    else:
        if score_dt>=score_rf:
            train_and_save_model(
            DecisionTreeRegressor(),
            X,
            y,
            'clean_data/best_model.pickle'
        )
        else:
            train_and_save_model(
            RandomForestRegressor(),
            X,
            y,
            'clean_data/best_model.pickle'
        )


  

In [54]:
find_score()

/home/formation/app_new/app_ven/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/formation/app_new/app_ven/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/formation/app_new/app_ven/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(9, 14)
LinearRegression() saved at  clean_data/best_model.pickle
